In [23]:
# 1단계: 필요한 패키지 설치
%uv pip install -q langchain-ollama

/Users/kyungseok.lee/workspace-git/langchain-basics-study/.venv/bin/python: No module named uv
Note: you may need to restart the kernel to use updated packages.


In [24]:
# 2단계: LLM 모델 초기화
from langchain_ollama import ChatOllama

llm = ChatOllama(model="gemma3:1b")
print("✅ LLM 모델이 초기화되었습니다.")

# Invalid input type <class 'int'>. Must be a PromptValue, str, or list of BaseMessages.
# llm.invoke(0)

✅ LLM 모델이 초기화되었습니다.


In [25]:
# 3단계: 기본 출력 파서 사용 예시
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate(
    template="What is the capital of {country}? Return the name of the city only",
    input_variables=["country"],
)

prompt = prompt_template.invoke({"country": "France"})
print("프롬프트:", prompt)

ai_message = llm.invoke(prompt_template.invoke({"country": "France"}))
print("AI 응답:", ai_message)

output_parser = StrOutputParser()
answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country": "France"})))
print("파싱된 답변:", answer)

프롬프트: text='What is the capital of France? Return the name of the city only'
AI 응답: content='Paris' additional_kwargs={} response_metadata={'model': 'gemma3:1b', 'created_at': '2025-09-05T16:56:04.251416Z', 'done': True, 'done_reason': 'stop', 'total_duration': 222118042, 'load_duration': 137292584, 'prompt_eval_count': 23, 'prompt_eval_duration': 72827875, 'eval_count': 2, 'eval_duration': 11521625, 'model_name': 'gemma3:1b'} id='run--874f5b6c-c87a-4c6f-847a-03b3c58d243b-0' usage_metadata={'input_tokens': 23, 'output_tokens': 2, 'total_tokens': 25}
파싱된 답변: Paris


In [26]:
ai_message

AIMessage(content='Paris', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-09-05T16:56:04.251416Z', 'done': True, 'done_reason': 'stop', 'total_duration': 222118042, 'load_duration': 137292584, 'prompt_eval_count': 23, 'prompt_eval_duration': 72827875, 'eval_count': 2, 'eval_duration': 11521625, 'model_name': 'gemma3:1b'}, id='run--874f5b6c-c87a-4c6f-847a-03b3c58d243b-0', usage_metadata={'input_tokens': 23, 'output_tokens': 2, 'total_tokens': 25})

In [27]:
answer

'Paris'

In [28]:
from pydantic import BaseModel, Field


class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")


structured_llm = llm.with_structured_output(CountryDetail)

In [29]:
from langchain_core.prompts import PromptTemplate

# from langchain_core.output_parsers import JsonOutputParser

country_detail_prompt = PromptTemplate(
    template="""Give following information about {country}:
    - Capital
    - Population
    - Language
    - Currency
    return it in JSON format. and return the JSON dictionary only
    """,
    input_variables=["country"],
)

# JsonOutputParser는 잘 동작하지 않음.
# country_detail_prompt.invoke({"country": "France"})
# output_parser = JsonOutputParser()
# print(json_ai_message.content)

json_ai_message = structured_llm.invoke(
    country_detail_prompt.invoke({"country": "France"})
)

# output_parser.invoke(json_ai_message)

In [30]:
(json_ai_message)

CountryDetail(capital='Paris', population=6773837, language='French', currency='Euro')

In [31]:
capital_chain = prompt_template | llm | output_parser

In [32]:
capital_chain.invoke({"country": "Korea"})

'Seoul'

In [33]:
country_prompt = PromptTemplate(
    template="""Guess the name of country based on the following information:
    {information}
    return the name of the country only
    """,
    input_variables=["information"],
)

country_chain = country_prompt | llm | output_parser
country_chain.invoke({"information": "This country is very famous for its wine"})

'France'

In [39]:
from langchain_core.runnables import RunnablePassthrough

# final_chain = {"country": country_chain} | capital_chain
# final_chain.invoke({"information": "This country is very famous for its wine"})

# final_chain = {"information": RunnablePassthrough()} | {"country": country_chain} | capital_chain
# final_chain.invoke("This country is very famous for its wine")

final_chain = (
    {"information": RunnablePassthrough(), "continent": RunnablePassthrough()}
    | {"country": country_chain}
    | capital_chain
)
final_chain.invoke(
    {"information": "This country is very famous for its wine", "continent": "Europe"}
)

'Paris'